# Cleaning a list of phone numbers

One of the most common tasks in Data Science, cleaning data and bringing it to a suitable format. When participants registered, they were giving their phone numbers in a number of different formats. We cleaned and sorted everything together, so that we would be able to communicate with them via SMS if necessary. 

Formats included:
* Phone numbers with symbols (e.g. 99999@)
* Phone numbers starting with the national code (e.g. +30 or 0030 222222)
* Phone numbers with slash (e.g. 2222233333 /22)
* Two or more phone numbers (e.g. 22200001199 or 22200033399)

Phone numbers will not be shown for anonymization purposes.

In [62]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_rows',5000)

In [64]:
df = pd.read_csv('Files/organization_participations-190910.csv',sep=';')

In [37]:
df.set_index('id',inplace=True)

In [38]:
d = df['phone'].to_frame()

In [40]:
a = [i for i in d]

'@', space, (), - , letters
i.e. keep only numbers

starting by 30 or 00: 12 or 14 digits
starting by 2 or 6: 10 digits
if >1 10-digit number: keep the one starting with 6

# Create valid regex

In [41]:
r_10_m = r'^69\d{8}' # 10 digits, mobile
r_10_l = r'^2\d{9}' # 10 digits, landline
r_14_m = r'^003069\d{8}' # 14 digits, mobile (incl national code)
r_14_l = r'^00302\d{9}'
r_13_m = r'^3069\d{8}' # 34 digits, mobile (incl national code)
r_13_l = r'^302\d{9}'
reg_list = [ r_10_m,r_10_l,  r_13_m,r_13_l,  r_14_m , r_14_l]
reg_names = ['10_mobile', '10_landline',  '13_mobile', '13_landline',  '14_mobile' , '14_landline']

In [42]:
d['phone_clear'] = d['phone'].str.replace('[^0-9]',"", regex=True)

# Fix concatenation

Get all phone numbers from all the entries. Some entries contained up to 3 different numbers.

In [44]:
phones = pd.DataFrame(index = d.index)

# 1st round

for i in reversed(range(len(reg_list))):
    reg = reg_list[i]
    print (reg_names[i])
    temp = pd.DataFrame([re.match(reg, i).group() if re.search(reg, i) else np.nan for i in d['phone_clear']])
    print (temp.shape)
    d['phone_clear'] = d['phone_clear'].str.replace(reg,'',regex=True,n=1)
    print(d[d['phone_clear']!=''].shape)
    temp.columns = np.array([reg_names[i]]*temp.shape[1])
    temp.index = d.index
    phones = pd.concat([phones, temp], axis=1,ignore_index = False, sort = False)

14_landline
(4776, 1)
(4619, 2)
14_mobile
(4776, 1)
(4615, 2)
13_landline
(4776, 1)
(4611, 2)
13_mobile
(4776, 1)
(4595, 2)
10_landline
(4776, 1)
(2276, 2)
10_mobile
(4776, 1)
(197, 2)


## For each participant, keep only one of the phone numbers we found, starting from 10_mobile if it exists and working our way upwards to 14_landline.

In [45]:
phones['final'] =np.nan

cols = phones.columns[:-1].tolist()
cols.reverse()

for c in cols:
    print (c)
    phones.loc[phones['final'].isna(),'final'] = phones.loc[phones['final'].isna(),c]
print(phones['final'].isna().sum())

10_mobile
10_landline
13_mobile
13_landline
14_mobile
14_landline
188


In [46]:
cols

['10_mobile',
 '10_landline',
 '13_mobile',
 '13_landline',
 '14_mobile',
 '14_landline']

In [47]:
d['final'] = phones['final']
d['id_final'] = phones.index
d['num_digits'] = [len(a) for a in d['phone']]

In [ ]:
a = d['id_final'] == d.index

In [50]:
# Check results
d_final = d.drop(columns = ['phone_clear', 'id_final']).sort_values(by='num_digits', ascending = False)

In [52]:
# Remove national code
r_0030 = r'^0030'
r_30 = r'^30'

a = d['final'].str.contains(r_0030,regex=True).fillna(value = False)
b = d['final'].str.contains(r_30,regex=True).fillna(value = False)

In [53]:
d['final'] = d['final'].str.replace(r_0030,'',regex=True)
d['final'] = d['final'].str.replace(r_30,'',regex=True)

In [54]:
# Fill NaNs for Database purposes
d['final'].fillna(value = '0000000000',inplace=True)

In [55]:
d['final_digits'] = [len(a) for a in d['final']]

And this is how a task that would have taken hours of manual effort, can be easily done in python.